In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
from peft import LoraConfig
import platform

The model that you want to train from the Hugging Face hub

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

The instruction dataset to use

In [ ]:
dataset_name = "codyburker/yelp_review_sampled"

Fine-tuned model name

In [ ]:
new_model = "DINO"

##############################################################################<br>
QLoRA parameters<br>
##############################################################################

LoRA attention dimension

In [ ]:
lora_r = 32  # Reduced from 64

Alpha parameter for LoRA scaling

In [ ]:
lora_alpha = 16

Dropout probability for LoRA layers

In [ ]:
lora_dropout = 0.1

##############################################################################<br>
bitsandbytes parameters<br>
##############################################################################

Activate 4-bit precision base model loading

In [ ]:
use_4bit = True

Compute dtype for 4-bit base models

In [ ]:
bnb_4bit_compute_dtype = "float16"

Quantization type (fp4 or nf4)

In [ ]:
bnb_4bit_quant_type = "nf4"

Activate nested quantization for 4-bit base models (double quantization)

In [ ]:
use_nested_quant = True  # Changed to True for further memory savings

##############################################################################<br>
TrainingArguments parameters<br>
##############################################################################

Output directory where the model predictions and checkpoints will be stored

In [ ]:
output_dir = "./results"

Number of training epochs

In [ ]:
num_train_epochs = 1

Enable fp16/bf16 training (set bf16 to True with an A100)

In [ ]:
fp16 = True  # Changed to True
bf16 = False  # Changed to False

Batch size per GPU for training

In [ ]:
per_device_train_batch_size = 1  # Reduced from 2

Batch size per GPU for evaluation

In [ ]:
per_device_eval_batch_size = 1  # Reduced from 2

Number of update steps to accumulate the gradients for

In [ ]:
gradient_accumulation_steps = 4  # Increased from 1

Enable gradient checkpointing

In [ ]:
gradient_checkpointing = True

Maximum gradient normal (gradient clipping)

In [ ]:
max_grad_norm = 0.3

Initial learning rate (AdamW optimizer)

In [ ]:
learning_rate = 1e-4  # Reduced from 2e-4

Weight decay to apply to all layers except bias/LayerNorm weights

In [ ]:
weight_decay = 0.001

Optimizer to use

In [ ]:
optim = "paged_adamw_32bit"

Learning rate schedule (constant a bit better than cosine)

In [ ]:
lr_scheduler_type = "constant"

Number of training steps (overrides num_train_epochs)

In [ ]:
max_steps = 100  # Added a limit to reduce overall training time

Ratio of steps for a linear warmup (from 0 to learning rate)

In [ ]:
warmup_ratio = 0.03

Group sequences into batches with same length<br>
Saves memory and speeds up training considerably

In [ ]:
group_by_length = True

Save checkpoint every X updates steps

In [ ]:
save_steps = 50  # Increased from 25

Log every X updates steps

In [ ]:
logging_steps = 50  # Increased from 25

##############################################################################<br>
SFT parameters<br>
##############################################################################

Maximum sequence length to use

In [ ]:
max_seq_length = 512  # Added a limit to reduce memory usage

Pack multiple short examples in the same input sequence to increase efficiency

In [ ]:
packing = False

Load the entire model on the GPU 0

In [ ]:
device_map = {"": 0}
mac_device_map = {"mps": 0}

Load dataset (you can process it here)

In [ ]:
dataset = load_dataset(dataset_name, split="train")

Load tokenizer and model with QLoRA configuration

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
if(not platform.system() == "Darwin"):
    # Check GPU compatibility with bfloat16
    if compute_dtype == torch.float16 and use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16: accelerate training with bf16=True")
            print("=" * 80)

Load base model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    max_memory={0:"2GB","cpu": "30GiB"},  # Reduced memory allocation
)
model.config.use_cache = False  # Changed to False to save memory
model.config.pretraining_tp = 1

Load LLaMA tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Load LoRA configuration

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Set training parameters

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

Set supervised fine-tuning parameters

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Train model

In [ ]:
trainer.train()

Save trained model

In [ ]:
trainer.model.save_pretrained(new_model)